# dependencies 

In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


# imports

In [48]:
import pandas as pd 
import numpy as np 

# using selenium scrape page

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
a
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

# Set up Chrome driver service
service = Service('/path/to/chromedriver')  # Replace with the actual path to chromedriver

# Start the Chrome webdriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Load the webpage
driver.get('https://bulbapedia.bulbagarden.net/wiki/EX_Crystal_Guardians_(TCG)#Set_lists')  # Replace with the actual URL of the webpage

# Wait for the page to load (you may need to adjust the waiting time depending on the page)
driver.implicitly_wait(10)  # Wait for 10 seconds

# Find the table element using the updated XPath
table_element = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/table[4]/tbody/tr/td[1]/table/tbody/tr[2]/td/table')

# Extract the "No.", card name, type, and rarity
no_element = table_element.find_element(By.XPATH, './/tbody/tr[2]/td[1]')
card_name_element = table_element.find_element(By.XPATH, './/tbody/tr[2]/td[3]/a')
card_type_elements = table_element.find_elements(By.XPATH, './/tbody/tr[2]/th/a/img')
card_rarity_element = table_element.find_element(By.XPATH, './/tbody/tr[2]/td[4]/a/img')

# Get the text content from the elements
card_no = no_element.text
card_name = card_name_element.text
card_types = [element.get_attribute('alt') for element in card_type_elements]
card_rarity = card_rarity_element.get_attribute('alt')

# Get the tables from the webpage using pd.read_html
tables = pd.read_html(driver.page_source)

# Assuming the table you want to update is the first table on the page
table = tables[14]

# Add the type and rarity to the respective row in the table
table['Type'] = ', '.join(card_types)
table['Rarity'] = card_rarity

# Print the updated table
print(table)

# Close the Chrome webdriver
driver.quit()


         No.    Card name     Type     Rarity
0      1/100      Banette  Psychic  Rare Holo
1      2/100  Blastoise δ  Psychic  Rare Holo
2      3/100     Camerupt  Psychic  Rare Holo
3      4/100  Charizard δ  Psychic  Rare Holo
4      5/100      Dugtrio  Psychic  Rare Holo
..       ...          ...      ...        ...
96    97/100   Shiftry ex  Psychic  Rare Holo
97    98/100  Swampert ex  Psychic  Rare Holo
98    99/100     Alakazam  Psychic  Rare Holo
99   100/100       Celebi  Psychic  Rare Holo
100      NaN          NaN  Psychic  Rare Holo

[101 rows x 4 columns]


<div class="alert alert-block alert-info">
<b>INFO:</b> 
    
* The method above did not work. The type and rarity were repeated but along with those issues, the trainer cards were also giving me some issue.  
   
* The next attempt is to turn it into soup and explore the html.  
    </div>

In [2]:
# create the soup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

# Set up Chrome driver service
service = Service('/path/to/chromedriver')  # Replace with the actual path to chromedriver

# Start the Chrome webdriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Load the webpage
driver.get('https://bulbapedia.bulbagarden.net/wiki/EX_Crystal_Guardians_(TCG)#Set_lists')  # Replace with the actual URL of the webpage

# Wait for the page to load (you may need to adjust the waiting time depending on the page)
driver.implicitly_wait(10)  # Wait for 10 seconds

# Find the table element using the XPath
table_element = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/table[4]/tbody/tr/td[1]/table/tbody/tr[2]/td/table')

# Get the HTML source of the table element
table_html = table_element.get_attribute('innerHTML')

# Create a BeautifulSoup object from the HTML source
soup = BeautifulSoup(table_html, 'html.parser')

# You can now use the 'soup' object to parse and extract data from the table

# Close the Chrome webdriver
driver.quit()


# Check Soup

In [15]:
soup


<tbody><tr style="background:#8CE6FF;">
<th style="border-top-left-radius: 6px; -moz-border-radius-topleft: 6px; -webkit-border-top-left-radius: 6px; -khtml-border-top-left-radius: 6px; -icab-border-top-left-radius: 6px; -o-border-top-left-radius: 6px;">No.
</th>
<th style="display:none;">Image
</th>
<th class="unsortable">Card name
</th>
<th class="unsortable" style="border-top-right-radius: 0px; -moz-border-radius-topright: 0px; -webkit-border-top-right-radius: 0px; -khtml-border-top-right-radius: 0px; -icab-border-top-right-radius: 0px; -o-border-top-right-radius: 0px;">Type
</th>
<th class="unsortable" style="border-top-right-radius: 6px; -moz-border-radius-topright: 6px; -webkit-border-top-right-radius: 6px; -khtml-border-top-right-radius: 6px; -icab-border-top-right-radius: 6px; -o-border-top-right-radius: 6px;">Rarity
</th>
<th class="unsortable" style="display:none; border-top-right-radius: 6px; -moz-border-radius-topright: 6px; -webkit-border-top-right-radius: 6px; -khtml-bor

# serial numbers

In [ ]:
# create a list of the serial numbers in a given expansion and set

In [16]:
numbers = soup.find_all('td', attrs={'align': 'center', 'style': 'background:#FFFFFF'})

In [17]:
numbers_filtered = [num.text.strip() for num in numbers if num.text.strip().endswith("/100")]

In [140]:
numbers_filtered[:5] # these are the first 5 serial #'s

['1/100', '2/100', '3/100', '4/100', '5/100']

# card names

In [ ]:
# the method to find names through html

In [157]:
# find all html that holds card names
items = soup.find_all('td', attrs={'style': 'background:#FFFFFF'})
# create an empty list
cardNames = []
#
for item in items:
    # avoiding useless html
    if item.find('a') and '/100' not in item.text:
        # strip the text to find root name
        card_name = item.find('a').text.strip()
        # add to list
        cardNames.append(card_name)

# filter out the list by booleans, if x is not a space then true; KEEP    
cardNames = list(filter(lambda x: x != '', cardNames))    

# types

In [166]:
types = soup.find_all('th', attrs={'align': 'center'})

In [167]:
pTypes = []

for i in range(len(types)):
    html = types[i]
    
    if 70 <= i <= 86:
        # Extract the energy title from the HTML
        title = html.text.strip()
        pTypes.append(title)  # Append HTML text to the main list
    
    else:
        # Ensure there are at least two 'a' tags for energy titles
        a_tags = html.find_all('a')
        num_a_tags = len(a_tags)
        
        if num_a_tags >= 2:
            type1 = a_tags[0]
            title1 = type1['title']
            
            type2 = a_tags[1]
            title2 = type2['title']
            
            pTypes.append([title1, title2])  # Append a list of two types to the main list
        elif num_a_tags == 1:
            a_tag = a_tags[0]
            title = a_tag['title']
            
            pTypes.append(title)  # Append single type to the main list
        else:
            print(f"Error: Insufficient 'a' tags for element {i}")


In [170]:
pTypes[:5]

['Psychic', ['Fighting', 'Metal'], 'Fire', ['Lightning', 'Metal'], 'Fighting']

# rarity

In [175]:
image_anchors = soup.find_all('a', class_='image')

image_anchors[1]['title']

# Extract the title starting at index 1 and skipping every other one
titles = [anchor['title'] for anchor in image_anchors[1::2] if 'title' in anchor.attrs]

possible_titles = ['Rare Holo','Rare','Uncommon','Common', 'Rare Holo ex', 'Shiny Rare', 'ShinyRare Holo']

# Extract the titles that match the specified values
Rarity = []
for anchor in image_anchors:
    title = anchor.get('title')
    if title in possible_titles:
        Rarity.append(title)


In [177]:
Rarity[:4]

['Rare Holo', 'Rare Holo', 'Rare Holo', 'Rare Holo']

# Creating A Dataframe

### serial num

In [178]:
len(numbers_filtered)

100

### card Names

In [179]:
len(cardNames)

100

### types

In [180]:
len(pTypes)

100

### rarity

In [181]:
len(Rarity)

100

In [182]:
numbers_filtered, cardNames, pTypes,Rarity

(['1/100',
  '2/100',
  '3/100',
  '4/100',
  '5/100',
  '6/100',
  '7/100',
  '8/100',
  '9/100',
  '10/100',
  '11/100',
  '12/100',
  '13/100',
  '14/100',
  '15/100',
  '16/100',
  '17/100',
  '18/100',
  '19/100',
  '20/100',
  '21/100',
  '22/100',
  '23/100',
  '24/100',
  '25/100',
  '26/100',
  '27/100',
  '28/100',
  '29/100',
  '30/100',
  '31/100',
  '32/100',
  '33/100',
  '34/100',
  '35/100',
  '36/100',
  '37/100',
  '38/100',
  '39/100',
  '40/100',
  '41/100',
  '42/100',
  '43/100',
  '44/100',
  '45/100',
  '46/100',
  '47/100',
  '48/100',
  '49/100',
  '50/100',
  '51/100',
  '52/100',
  '53/100',
  '54/100',
  '55/100',
  '56/100',
  '57/100',
  '58/100',
  '59/100',
  '60/100',
  '61/100',
  '62/100',
  '63/100',
  '64/100',
  '65/100',
  '66/100',
  '67/100',
  '68/100',
  '69/100',
  '70/100',
  '71/100',
  '72/100',
  '73/100',
  '74/100',
  '75/100',
  '76/100',
  '77/100',
  '78/100',
  '79/100',
  '80/100',
  '81/100',
  '82/100',
  '83/100',
  '84/100',
 

In [183]:
data = {'serial_number': numbers_filtered,
        'name': cardNames,
        'type': pTypes,
        'rarity':Rarity}

In [184]:
cards = pd.DataFrame(data)

In [185]:
cards["set"] = "Crystal Guardians"

In [186]:
cards

,serial_number,name,type,rarity,set
0,1/100,Banette,Psychic,Rare Holo,Crystal Guardians
1,2/100,Blastoise δ,"[Fighting, Metal]",Rare Holo,Crystal Guardians
2,3/100,Camerupt,Fire,Rare Holo,Crystal Guardians
3,4/100,Charizard δ,"[Lightning, Metal]",Rare Holo,Crystal Guardians
4,5/100,Dugtrio,Fighting,Rare Holo,Crystal Guardians
...,...,...,...,...,...
95,96/100,Sceptile ex δ,Psychic,Rare Holo ex,Crystal Guardians
96,97/100,Shiftry ex,Darkness,Rare Holo ex,Crystal Guardians
97,98/100,Swampert ex,Water,Rare Holo ex,Crystal Guardians
98,99/100,Alakazam,Psychic,ShinyRare Holo,Crystal Guardians


# moving to sql

In [54]:
# importing credentials
import env

In [187]:
import mysql.connector
import os

# Retrieve the password from the environment variable
password = env.password
username = env.username

# Establish a connection
connection = mysql.connector.connect(
    host='localhost',
    user= username,
    password=password,
    database='PokemonTCG'
)

# Check if the connection is successful
if connection.is_connected():
    print("Connected to MySQL database")

# Perform database operations
# ...

# Close the connection when finished
connection.close()


Connected to MySQL database


# testing the connection

In [189]:
import mysql.connector

# Establish a connection
connection = mysql.connector.connect(
    host='localhost',
    user= username,
    password= password,
    database='PokemonTCG'
)

# Check if the connection is successful
if connection.is_connected():
    print("Connected to MySQL database")

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Execute a simple SELECT query
cursor.execute("SELECT * FROM expansions")

# Fetch the rows from the result set
rows = cursor.fetchall()

rows

# Close the cursor and connection
cursor.close()
connection.close()


Connected to MySQL database


In [190]:
rows

[(1, 'Scarlet and Violet'),
 (2, 'Sword and Shield'),
 (3, 'Sun and Moon'),
 (4, 'XY'),
 (5, 'Black and White'),
 (6, 'Call of Legends and HGSS Series'),
 (7, 'Platinum'),
 (8, 'Diamond and Pearl'),
 (9, 'EX')]

# Send Serial Number to SQL

In [191]:
import mysql.connector
import pandas as pd

# Establish a connection
connection = mysql.connector.connect(
    host='localhost',
    user= username,
    password= password,
    database='PokemonTCG'
)

# Check if the connection is successful
if connection.is_connected():
    print("Connected to MySQL database")

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Get the 'serial_number' values from the DataFrame
serial_numbers = cards['serial_number'].tolist()

# Insert the 'serial_number' values into the 'cards' table
for serial_number in serial_numbers:
    query = "INSERT INTO cards (serial_number) VALUES (%s)"
    values = (serial_number,)
    #print(query)
    #print(values)
    cursor.execute(query, values)

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()


Connected to MySQL database


# send card_name to SQl

In [192]:
for serial_number, card_name in zip(serial_numbers, card_names):
    print(serial_number,card_name)
    print('\n')
    

1/100 Banette


2/100 Blastoise δ


3/100 Camerupt


4/100 Charizard δ


5/100 Dugtrio


6/100 Ludicolo δ


7/100 Luvdisc


8/100 Manectric


9/100 Mawile


10/100 Sableye


11/100 Swalot


12/100 Tauros


13/100 Wigglytuff


14/100 Blastoise


15/100 Cacturne δ


16/100 Combusken


17/100 Dusclops


18/100 Fearow δ


19/100 Grovyle δ


20/100 Grumpig


21/100 Igglybuff


22/100 Kingler δ


23/100 Loudred


24/100 Marshtomp


25/100 Medicham


26/100 Pelipper δ


27/100 Swampert


28/100 Venusaur


29/100 Charmeleon


30/100 Charmeleon δ


31/100 Combusken


32/100 Grovyle


33/100 Gulpin


34/100 Ivysaur


35/100 Ivysaur


36/100 Lairon


37/100 Lombre


38/100 Marshtomp


39/100 Nuzleaf


40/100 Shuppet


41/100 Skitty


42/100 Wartortle


43/100 Wartortle


44/100 Aron


45/100 Bulbasaur


46/100 Bulbasaur


47/100 Cacnea


48/100 Charmander


49/100 Charmander δ


50/100 Diglett


51/100 Duskull


52/100 Electrike


53/100 Jigglypuff


54/100 Krabby


55/100 Lotad


56/100 Meditite

In [193]:
# Establish a connection
connection = mysql.connector.connect(
    host='localhost',
    user= username,
    password= password,
    database='PokemonTCG'
)

# Check if the connection is successful
if connection.is_connected():
    print("Connected to MySQL database")

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Get the serial numbers and card names from the DataFrame
card_names = cards['name'].tolist()

# Iterate over the serial numbers and card names to update the table
for serial_number, card_name in zip(serial_numbers, card_names):
    update_query = "UPDATE cards SET card_name = %s WHERE serial_number = %s"
    values = (card_name, serial_number)
    cursor.execute(update_query, values)

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()


Connected to MySQL database


# adding Rarity to SQL

<div class= "alert alert-block alert-info">
    <b> INFO: </b>
    Keep in mind that there is a foreign key present of rarity
</div>

In [119]:
# Iterate over the rows in the DataFrame
for index, row in cards.iterrows():
    serial_number = row['serial_number']
    card_name = row['name']
    rarity = row['rarity']
    
    print(serial_number, card_name, rarity)

1/100 Banette Rare Holo
2/100 Blastoise δ Rare Holo
3/100 Camerupt Rare Holo
4/100 Charizard δ Rare Holo
5/100 Dugtrio Rare Holo
6/100 Ludicolo δ Rare Holo
7/100 Luvdisc Rare Holo
8/100 Manectric Rare Holo
9/100 Mawile Rare Holo
10/100 Sableye Rare Holo
11/100 Swalot Rare Holo
12/100 Tauros Rare Holo
13/100 Wigglytuff Rare Holo
14/100 Blastoise Rare
15/100 Cacturne δ Rare
16/100 Combusken Rare
17/100 Dusclops Rare
18/100 Fearow δ Rare
19/100 Grovyle δ Rare
20/100 Grumpig Rare
21/100 Igglybuff Rare
22/100 Kingler δ Rare
23/100 Loudred Rare
24/100 Marshtomp Rare
25/100 Medicham Rare
26/100 Pelipper δ Rare
27/100 Swampert Rare
28/100 Venusaur Rare
29/100 Charmeleon Uncommon
30/100 Charmeleon δ Uncommon
31/100 Combusken Uncommon
32/100 Grovyle Uncommon
33/100 Gulpin Uncommon
34/100 Ivysaur Uncommon
35/100 Ivysaur Uncommon
36/100 Lairon Uncommon
37/100 Lombre Uncommon
38/100 Marshtomp Uncommon
39/100 Nuzleaf Uncommon
40/100 Shuppet Uncommon
41/100 Skitty Uncommon
42/100 Wartortle Uncommon
4

In [194]:
# Establish a connection
connection = mysql.connector.connect(
    host='localhost',
    user= username,
    password= password,
    database='PokemonTCG'
)

# Check if the connection is successful
if connection.is_connected():
    print("Connected to MySQL database")

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Iterate over the rows in the DataFrame
for index, row in cards.iterrows():
    serial_number = row['serial_number']
    card_name = row['name']
    rarity = row['rarity']

    # Retrieve the rarity_id from the rarity table
    select_query = "SELECT rarity_id FROM rarity WHERE rarity_name = %s"
    cursor.execute(select_query, (rarity,))
    result = cursor.fetchone()

    if result is not None:
        rarity_id = result[0]

        # Update the cards table with the rarity_id
        update_query = "UPDATE cards SET rarity_id = %s WHERE serial_number = %s AND card_name = %s"
        update_values = (rarity_id, serial_number, card_name)
        cursor.execute(update_query, update_values)

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()


Connected to MySQL database


# Adding set to SQL

<div class= 'alert alert-block alert-info'>
    keep in mind the foreign key
</div>

In [195]:
# Establish a connection
connection = mysql.connector.connect(
    host='localhost',
    user= username,
    password= password,
    database='PokemonTCG'
)
# Check if the connection is successful
if connection.is_connected():
    print("Connected to MySQL database")

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Iterate over the rows in the DataFrame
for index, row in cards.iterrows():
    serial_number = row['serial_number']
    card_name = row['name']
    rarity = row['rarity']
    set_name = row['set']

    # Retrieve the set_id from the sets table
    select_query = "SELECT set_id FROM sets WHERE set_name = %s"
    cursor.execute(select_query, (set_name,))
    result = cursor.fetchone()

    if result is not None:
        set_id = result[0]

        # Retrieve the rarity_id from the rarity table
        select_rarity_query = "SELECT rarity_id FROM rarity WHERE rarity_name = %s"
        cursor.execute(select_rarity_query, (rarity,))
        rarity_result = cursor.fetchone()

        if rarity_result is not None:
            rarity_id = rarity_result[0]

            # Update the cards table with the set_id and rarity_id
            update_query = "UPDATE cards SET set_id = %s, rarity_id = %s WHERE serial_number = %s AND card_name = %s"
            update_values = (set_id, rarity_id, serial_number, card_name)
            cursor.execute(update_query, update_values)

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()


Connected to MySQL database


# Practice using fetchone

In [196]:
# Establish a connection
connection = mysql.connector.connect(
    host='localhost',
    user= username,
    password= password,
    database='PokemonTCG'
)

# Check if the connection is successful
if connection.is_connected():
    print("Connected to MySQL database")

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

cursor.execute("SELECT * FROM cards")
row = cursor.fetchone()

while row is not None:
    # Process the row
    print(row)

    # Fetch the next row
    row = cursor.fetchone()


Connected to MySQL database
(201, '1/100', 'Banette', 4, 97)
(202, '2/100', 'Blastoise δ', 4, 97)
(203, '3/100', 'Camerupt', 4, 97)
(204, '4/100', 'Charizard δ', 4, 97)
(205, '5/100', 'Dugtrio', 4, 97)
(206, '6/100', 'Ludicolo δ', 4, 97)
(207, '7/100', 'Luvdisc', 4, 97)
(208, '8/100', 'Manectric', 4, 97)
(209, '9/100', 'Mawile', 4, 97)
(210, '10/100', 'Sableye', 4, 97)
(211, '11/100', 'Swalot', 4, 97)
(212, '12/100', 'Tauros', 4, 97)
(213, '13/100', 'Wigglytuff', 4, 97)
(214, '14/100', 'Blastoise', 3, 97)
(215, '15/100', 'Cacturne δ', 3, 97)
(216, '16/100', 'Combusken', 3, 97)
(217, '17/100', 'Dusclops', 3, 97)
(218, '18/100', 'Fearow δ', 3, 97)
(219, '19/100', 'Grovyle δ', 3, 97)
(220, '20/100', 'Grumpig', 3, 97)
(221, '21/100', 'Igglybuff', 3, 97)
(222, '22/100', 'Kingler δ', 3, 97)
(223, '23/100', 'Loudred', 3, 97)
(224, '24/100', 'Marshtomp', 3, 97)
(225, '25/100', 'Medicham', 3, 97)
(226, '26/100', 'Pelipper δ', 3, 97)
(227, '27/100', 'Swampert', 3, 97)
(228, '28/100', 'Venusaur',